In [1]:
!uv pip install boto3
!uv pip install sagemaker

Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 9ms
Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 25ms


In [2]:
import os
import json
import logging
from datetime import datetime

In [3]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\bored\AppData\Local\sagemaker\sagemaker\config.yaml


In [4]:
sess = sagemaker.Session()
region = sess.boto_region_name
print(region)

us-east-1


In [5]:
import boto3

iam_client = boto3.client('iam')


role_name = "SageMakerExecutionRole"

response = iam_client.get_role(RoleName=role_name)

role_arn = response['Role']['Arn']

print(role_arn)


arn:aws:iam::396913733336:role/SageMakerExecutionRole


In [7]:
import requests

url = 'https://github.com/KannamSridharKumar/AWS_Sagemaker_Course/raw/main/image_classification_datasets.zip'
response = requests.get(url)

with open('image_classification_datasets.zip', 'wb') as f:
    f.write(response.content)


In [15]:
import zipfile
with zipfile.ZipFile('image_classification_datasets.zip', 'r') as zip_ref:
    zip_ref.extractall('image_classification_datasets')

In [23]:

bucket_name = 'sgmkr-learner'
data_path = "sgmkr_clf_lst"

nclasses = 3
nimgs_train = 36
nepochs = 10
mini_batch_size = 8

train_instance_type = "ml.m5.large"
job_name_prefix = "flowers-clf-ib-resent-"

In [9]:
train_image_uri = sagemaker.image_uris.retrieve(
    framework="image-classification",
    region=region,
    image_scope="training",
    version="latest",
)
print(train_image_uri)

811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [10]:
s3_output_path = "s3://{}/{}/{}".format(bucket_name, data_path, "model_output")


In [24]:
clf_estimator = sagemaker.estimator.Estimator(
    image_uri=train_image_uri,
    role=role_arn,
    instance_count=1,
    instance_type=train_instance_type,
    volume_size=50,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_path,
    sagemaker_session=sess,
)

In [25]:
clf_estimator.set_hyperparameters(
    num_classes=nclasses,  # update this
    epochs=nepochs,  # update this
    num_training_samples=nimgs_train,  # update this
    mini_batch_size=mini_batch_size,  # update this
    num_layers=18,
    use_pretrained_model=1,
    image_shape="3,224,224",
    resize=256,
    learning_rate=0.001,
    use_weighted_loss=1,
    augmentation_type="crop_color_transform",
    precision_dtype="float32",
    multi_label=0,
)

In [ ]:
bucket_name = "sgmkr-course"
data_path = "sgmkr_clf_lst"

In [19]:
# Upload train_annots
!aws s3 cp image_classification_datasets/image_classification_datasets/img_clf_lst/train_annots/ s3://{bucket_name}/{data_path}/train_annots/ --recursive --exclude "*" --include "*.jpg"

# Upload train_imgs
!aws s3 cp image_classification_datasets/image_classification_datasets/img_clf_lst/train_imgs/ s3://{bucket_name}/{data_path}/train_imgs/ --recursive --exclude "*" --include "*.jpg"

# Upload valid_annots
!aws s3 cp image_classification_datasets/image_classification_datasets/img_clf_lst/valid_annots/ s3://{bucket_name}/{data_path}/valid_annots/ --recursive --exclude "*" --include "*.jpg"

# Upload valid_imgs
!aws s3 cp image_classification_datasets/image_classification_datasets/img_clf_lst/valid_imgs/ s3://{bucket_name}/{data_path}/valid_imgs/ --recursive --exclude "*" --include "*.jpg"


Completed 19.1 KiB/1.8 MiB (5.6 KiB/s) with 99 file(s) remaining
upload: image_classification_datasets\image_classification_datasets\img_clf_lst\train_imgs\2120321.jpg to s3://sgmkr-learner/sgmkr_clf_lst/train_imgs/2120321.jpg
Completed 19.1 KiB/1.8 MiB (5.6 KiB/s) with 98 file(s) remaining
Completed 38.2 KiB/1.8 MiB (11.1 KiB/s) with 98 file(s) remaining
upload: image_classification_datasets\image_classification_datasets\img_clf_lst\train_imgs\2009869.jpg to s3://sgmkr-learner/sgmkr_clf_lst/train_imgs/2009869.jpg
Completed 38.2 KiB/1.8 MiB (11.1 KiB/s) with 97 file(s) remaining
Completed 57.3 KiB/1.8 MiB (16.6 KiB/s) with 97 file(s) remaining
upload: image_classification_datasets\image_classification_datasets\img_clf_lst\train_imgs\1715218.jpg to s3://sgmkr-learner/sgmkr_clf_lst/train_imgs/1715218.jpg
Completed 57.3 KiB/1.8 MiB (16.6 KiB/s) with 96 file(s) remaining
Completed 76.4 KiB/1.8 MiB (21.6 KiB/s) with 96 file(s) remaining
upload: image_classification_datasets\image_classifica

In [20]:
s3_train_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "train_imgs")
s3_valid_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_imgs")
s3_train_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "train_annots")
s3_valid_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_annots")

train_imgs = sagemaker.inputs.TrainingInput(
    s3_train_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_imgs = sagemaker.inputs.TrainingInput(
    s3_valid_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
train_annot = sagemaker.inputs.TrainingInput(
    s3_train_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_annot = sagemaker.inputs.TrainingInput(
    s3_valid_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)

data_channels = {
    "train": train_imgs,
    "validation": valid_imgs,
    "train_lst": train_annot,
    "validation_lst": valid_annot,
}

In [26]:
timestamp = (
    str(datetime.now().replace(microsecond=0)).replace(" ", "-").replace(":", "-")
)
job_name = job_name_prefix + timestamp
print(job_name)

flowers-clf-ib-resent-2025-06-09-22-12-53


In [27]:
clf_estimator.fit(inputs=data_channels, logs=True, job_name=job_name)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: flowers-clf-ib-resent-2025-06-09-22-12-53
ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 clf_estimator.fit(inputs=data_channels, logs=True, job_name=job_name)                        │
│   2                                                                                              │
│                                                                                                  │
│ c:\Users\bored\Music\SagemekerLearning\sagelearner\lib\site-packages\sagemaker\telemetry\telemet │
│ ry_logging.py:167 in wrapper                                                                     │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ c:\Users\bored\Music\SagemekerLearning\sagelearner\lib\site-packages\sagemaker\telemetry\telemet │
│ ry_logging.py:138 in wrapper                                                                     │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ c:\Users\bored\Music\SagemekerLearning\sagelearner\lib\site-packages\sagemaker\workflow\pipeline │
│ _context.py:346 in wrapper                                                                       │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ c:\Users\bored\Music\SagemekerLearning\sagelearner\lib\site